# CS 6140 Machine Learning: Final Group Project
## Philip Butler, Riley Grant, Benjamin Ruzzo

words words words

In [13]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib
import seaborn as sbn

words words words

In [ ]:
# code code code